In [7]:
import qutip as qt
from qutip.tensor import tensor
import numpy as np

def applyDDimenDepolNoise(rho: qt.Qobj, p: float) -> qt.Qobj:
    '''
    Apply depolarizing noise to a given quantum object.

    Parameters:
    - rho (qutip.Qobj): input state
    - p (float): The depolarizing probability, ranging from 0 to 1.

    Returns:
    - qutip.Qobj: The quantum object after applying depolarizing noise.
    '''
    assert p>=0 and p<=1, "The error rate p should be between 0 and 1."
    num_qubits = int(np.log2(rho.shape[0]))  # Determine the number of qubits

    rhoprime=(1-p)*rho+(p/(2**num_qubits)*qt.qeye(2**num_qubits))
    return rhoprime

def _extend_op(op: qt.Qobj, qubit_idx: int, num_qubits: int) -> qt.Qobj:
    '''Extends op to a larger subspace.
    
    Parameters:
    -op (qutip.Qobj): the operation to be extended.
    -qubit_idx (int): the qubit index where the operation should be applied.
    -num_qubits (int): total number of qubits.'''
    I=qt.qeye(2)
    ops=[I for idx in range(num_qubits) if idx!=qubit_idx]
    ops.insert(qubit_idx, op)
    final_op=ops[0]
    for elem in ops[1:]:
        final_op=tensor(final_op, elem)
    return final_op
    

def applySingleDepolNoise(rho: qt.Qobj, qubit_idx: int, p_x: float, p_y: float, p_z: float) -> qt.Qobj:
    '''
    Applies single qubit asymmetric depolarizing map in Kraus representation.

    Parameters:
    - rho (qutip.Qobj): input state
    - qubit_idx (int): qubit index to apply the depolarizing channel to.
    - p_x (float): x_error
    - p_y (float): y_error
    - p_z (float): z_error

    Returns:
    - qutip.Qobj: The quantum object after applying depolarizing noise.
    '''
    p_tot=p_x+p_y+p_z
    assert p_tot<=1 and p_tot>=0 and p_x>=0 and p_y>=0 and p_z>=0, "The error rate p should be between 0 and 1."
    I=qt.qeye(2)
    x=qt.sigmax()
    y=qt.sigmay()
    z=qt.sigmaz()
    print("type: ", y)
    num_qubits= int(np.log2(rho.shape[0]))
    # There is only one qubit so we can skip extending.
    if num_qubits==1:
        rhoPrime=(1-p_tot)*rho+(p_x*x*rho*x+p_y*y*rho*y+p_z*z*rho*z)
    else:
        x_extend=_extend_op(x, qubit_idx, num_qubits)
        y_extend=_extend_op(y, qubit_idx, num_qubits)
        z_extend=_extend_op(z, qubit_idx, num_qubits)
        rhoPrime=(1-p_tot)*rho+(p_x*x_extend*rho*x_extend+p_y*y_extend*rho*y_extend+p_z*z_extend*rho*z_extend)
    return rhoPrime


# Example usage:
initial_state = qt.basis(2, 0)
rho=initial_state * initial_state.dag()

# Apply depolarizing noise with probability 0.1
noisy_state = applyDDimenDepolNoise(rho, 0.1)

print("Initial State:")
print(initial_state)
print("\nNoisy State:")
print(noisy_state)
print(applySingleDepolNoise(rho,0, .1,.1,.8))
print(applySingleDepolNoise(tensor(rho, qt.qeye(2)),0, .1,.1,.8))


Initial State:
Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]

Noisy State:
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.95 0.  ]
 [0.   0.05]]
type:  Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.8 0. ]
 [0.  0.2]]
type:  Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.8 0.  0.  0. ]
 [0.  0.8 0.  0. ]
 [0.  0.  0.2 0. ]
 [0.  0.  0.  0.2]]
